In [34]:
from scipy.spatial.distance import cosine
import numpy as np
import pandas as pd
from PIL import Image
from sklearn import metrics as m
from tqdm import tqdm
import pickle
import os
import torch
from catboost import CatBoostClassifier,Pool,cv
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:

from autogluon.tabular import TabularDataset, TabularPredictor

In [30]:
df = pd.read_json('clip_embed.json')
embeddings_head = {}
for val in pickle.load(open('head2_embeddings.pkl', 'rb')).values():
    for data in val:
        #if data[0].startswith():
        #    continue
        key = data[0].replace('yolo_preds_swan_no_classify/', '').replace('malyi', 'разметка_малый').replace('shipun', 'разметка_шипун').split('/')
        key = key[0] + '/' + key[1][2:]
        embeddings_head[key] = embeddings_head.get(key, []) + [data[1]]

In [ ]:
train = []
for val in pickle.load(open('head2_embeddings.pkl', 'rb')).values():
    for data in val:
        #if data[0].startswith():
        #    continue
        key = data[0].replace('yolo_preds_swan_no_classify/', '').replace('malyi', 'разметка_малый').replace('shipun', 'разметка_шипун').split('/')
        key = key[0] + '/' + key[1][2:]
        row = [0 if 'малый' in key else 1 if 'klikun' in key else 2]
        row.extend(data[1])
        train.append(row)

In [ ]:
train = []
for val in pickle.load(open('swan_embeddings.pkl', 'rb')).values():
    for key, embeds in val:
        #if data[0].startswith():
        #    continue
        key = key.replace('yolo_preds_swan_no_classify/', '').replace('malyi', 'разметка_малый').replace('shipun', 'разметка_шипун').split('/')
        row = [0 if 'малый' in key else 1 if 'klikun' in key else 2]
        row.extend(embeds)
        train.append(row)

In [78]:
df

,cls,indexes,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,...,embed_758,embed_759,embed_760,embed_761,embed_762,embed_763,embed_764,embed_765,embed_766,embed_767
0,1,/kaggle/working/klikun/images/3465518321.jpg,-0.491297,0.472260,-0.792846,-0.091666,0.185589,-0.158303,0.216689,-0.437683,...,0.421369,-0.008260,-0.288490,0.349580,-0.261433,0.644165,0.133460,-0.506689,0.570624,-0.126007
1,1,/kaggle/working/klikun/images/3310013493.jpg,-0.037389,0.265436,-0.770407,-0.174408,-0.076229,-0.437367,0.560788,0.022536,...,0.636512,0.302980,-0.153714,-0.139685,-0.057687,0.126880,0.257803,0.067096,0.053801,0.543492
2,1,/kaggle/working/klikun/images/1835883276.jpg,-0.574928,0.984862,-0.373945,1.055974,0.152147,-0.482239,-0.002151,-0.602424,...,0.611408,0.265577,-0.248756,0.318295,-0.044623,0.693198,-0.085975,-0.595453,0.357164,0.177577
3,1,/kaggle/working/klikun/images/3499368095.jpg,-0.212149,0.436835,-0.825643,0.156009,-0.379423,-0.392241,0.144879,-0.136477,...,0.366045,0.054078,-0.083700,-0.046146,0.029196,0.652838,-0.036750,-0.489450,0.469299,-0.024670
4,1,/kaggle/working/klikun/images/1823296248.jpg,-0.069744,0.836027,-0.642897,0.048499,-0.254285,-0.161485,0.071634,-0.321019,...,0.134172,-0.008150,-0.478905,-0.048906,-0.711810,0.505342,-0.137379,-0.460576,0.525144,0.065985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8959,0,/kaggle/working/разметка_шипун/images/img_2080.jpg,-0.217860,0.489948,-0.503568,-0.027640,-0.810208,-0.094883,0.050743,-0.057113,...,0.311398,0.207909,-0.017362,-0.443973,-0.023800,0.203145,-0.036747,-0.385659,-0.125250,-0.072202
8960,0,/kaggle/working/разметка_шипун/images/img_2145.jpg,-0.801452,0.714187,-0.764315,-0.425820,-0.037532,-0.211331,-0.414045,-0.441321,...,0.400780,0.021274,-0.423666,0.141602,0.153813,0.423385,-0.111982,-0.581735,0.407517,0.362991
8961,0,/kaggle/working/разметка_шипун/images/img_2991.jpg,-0.256728,0.573361,-0.535171,-0.026079,-0.021683,-0.695079,0.202662,-0.490299,...,0.502922,0.617335,-0.348519,-0.427323,0.713674,-0.486096,0.031373,-0.544337,0.296180,0.117189
8962,0,/kaggle/working/разметка_шипун/images/img_1067.jpg,-0.090077,0.315957,-0.844520,-0.065187,-0.019200,-0.143684,-0.367889,-0.188275,...,0.531824,0.155172,-0.231552,-0.665782,-0.058494,-0.029696,0.101339,-0.670886,0.042258,0.115012


In [29]:
data[0].replace('yolo_preds_swan_no_classify/', '').replace('malyi', 'разметка_малый').replace('shipun', 'разметка_шипун')

'klikun/0_1.jpg'

In [68]:
train = []
for row in tqdm(list(df.iterrows())):
    row = row[1]
    cls, indexes = row[['cls', 'indexes']]
    indexes = indexes.split('/')[-3] + '/' + indexes.split('/')[-1]
    
    embeds1 = [row[f'embed_{i}'] for i in range(768)]
    try:
        embeds2 = np.array(embeddings_head[indexes]).mean(axis=0).tolist()
    except KeyError:
        continue
    
    cls = int(cls)
    train_row = [0 if cls == 2 else 1 if cls == 1 else 2]
    for i, emb in enumerate(embeds1 + embeds2, start=1):
        train_row.append(emb)
        
    train.append(train_row)

100%|██████████| 8964/8964 [00:33<00:00, 265.12it/s]


In [ ]:
train = []
for row in tqdm(list(df.iterrows())):
    row = row[1]
    cls, _ = row[['cls', 'indexes']]
    
    embeds = [row[f'embed_{i}'] for i in range(768)]
    
    cls = int(cls)
    train_row = [0 if cls == 2 else 1 if cls == 1 else 2]
    for emb in embeds:
        train_row.append(emb)
        
    train.append(train_row)

In [69]:
super_embed = pd.DataFrame(train, columns=['label'] + [f'embed_{i}' for i in range(768 * 2)])

In [75]:
train_data, test_data = train_test_split(super_embed, train_size=0.9, random_state=42)

In [74]:
super_embed.drop(['label'], axis=1)

,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_1526,embed_1527,embed_1528,embed_1529,embed_1530,embed_1531,embed_1532,embed_1533,embed_1534,embed_1535
0,-0.491297,0.472260,-0.792846,-0.091666,0.185589,-0.158303,0.216689,-0.437683,-0.058173,-0.748734,...,0.058888,0.279817,0.311138,0.089053,0.291840,-0.113352,-0.295876,-0.745563,0.479233,-0.562575
1,-0.037389,0.265436,-0.770407,-0.174408,-0.076229,-0.437367,0.560788,0.022536,-0.202502,-0.589956,...,-0.180323,0.057821,0.704642,0.300993,-0.372622,-0.504926,-0.348187,-0.334883,-0.172343,-0.344268
2,-0.574928,0.984862,-0.373945,1.055974,0.152147,-0.482239,-0.002151,-0.602424,0.054246,-0.712152,...,0.606275,0.244941,0.157968,-0.230411,0.334965,-0.099347,-0.322740,-0.382832,0.294898,-0.331612
3,-0.212149,0.436835,-0.825643,0.156009,-0.379423,-0.392241,0.144879,-0.136477,-0.189964,-0.594914,...,0.289842,0.174346,0.295127,-0.225782,0.369814,-0.141798,0.148121,-0.700180,0.207785,-0.454138
4,-0.069744,0.836027,-0.642897,0.048499,-0.254285,-0.161485,0.071634,-0.321019,0.363917,-1.103625,...,0.017860,0.269168,0.461906,0.034652,0.524163,-0.300850,0.074907,-0.620467,0.404612,-0.478757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8954,-0.217860,0.489948,-0.503568,-0.027640,-0.810208,-0.094883,0.050743,-0.057113,-0.028263,-0.169902,...,0.105788,0.066201,0.389316,0.091082,-0.120106,0.295003,-0.015712,-0.098874,0.398685,-0.315762
8955,-0.801452,0.714187,-0.764315,-0.425820,-0.037532,-0.211331,-0.414045,-0.441321,-0.102088,-0.501280,...,-0.240332,0.409869,0.563308,0.166342,0.067497,-0.114470,-0.141723,-0.684282,0.378853,-0.367898
8956,-0.256728,0.573361,-0.535171,-0.026079,-0.021683,-0.695079,0.202662,-0.490299,0.074990,-0.289745,...,0.042712,0.453859,0.048415,-0.084667,0.673482,-0.410677,0.025665,-0.556069,0.439336,-0.594239
8957,-0.090077,0.315957,-0.844520,-0.065187,-0.019200,-0.143684,-0.367889,-0.188275,-0.159656,-0.332594,...,-0.381164,0.501272,0.345498,0.441090,-0.081305,-0.179239,0.113484,-0.167868,-0.156689,0.079240


In [ ]:
def calculate_metric(y_true,y_pred):
    y_true = list(y_true)
    y_pred = list(y_pred)
    n1, n2, n3 = 0, 0, 0
    for y_t,y_p in zip(y_true,y_pred):
        if y_t == y_p == 0:
            n1 += 3
        if y_t == 0 and y_p == 1:
            n1 -= 1
        if y_t == 0 and y_p == 2:
            n1 -= 3
            
        if y_t == y_p == 1:
            n2 += 3
        if y_t == 1 and y_p == 0:
            n2 -= 1
        if y_t == 1 and y_p == 2:
            n2 -= 3
            
        if y_t == y_p == 2:
            n3 += 2
        if y_t == 2 and y_p == 0:
            n3 -= 3
        if y_t == 2 and y_p == 1:
            n3 -= 3
    max_res = y_true.count(0)*3 + y_true.count(1)*3 + y_true.count(2)*2
    return (n1 + n2 + n3) / max_res

In [80]:
hyperparameters = {
'NN_TORCH': {}, 
#'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge',],
'CAT': {}, 
'XGB': {}, 
'FASTAI': {},
'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, 
{'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, 
{'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}},], 
'XT': [
{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}},
],
'KNN': [
{'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}}, {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}},
],
}

In [ ]:
from autogluon.core.metrics import make_scorer
ag_our_metric_scorer = make_scorer(name='custom_metric',
                                 score_func=calculate_metric,
                                 optimum=1,
                                 greater_is_better=True)
predictor = TabularPredictor(label='label',eval_metric=ag_our_metric_scorer, problem_type="multiclass").fit(train_data,num_gpus=1, hyperparameters=hyperparameters)

In [ ]:
predictor = TabularPredictor(label='label',eval_metric='f1_weighted', problem_type="multiclass").fit(train_data,num_gpus=1, hyperparameters=hyperparameters)

In [77]:
predictor = TabularPredictor(label='label', eval_metric='f1_weighted').fit(train_data, num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels\ag-20230520_204137\"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230520_204137\"
AutoGluon Version:  0.7.0
Python Version:     3.10.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Train Data Rows:    7167
Train Data Columns: 1536
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	3 unique label values:  [1, 2, 0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Trai

KeyboardInterrupt: 

In [ ]:
predictor.leaderboard(test_data)

                  model  score_test  score_val  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2    0.946330   0.958141        2.095053       1.040706  367.465576                 0.006284                0.002449           2.216262            2       True         14
1        NeuralNetTorch    0.945780   0.949738        0.043614       0.029637   21.725185                 0.043614                0.029637          21.725185            1       True         12
2       NeuralNetFastAI    0.940793   0.952564        0.155018       0.039688   15.422325                 0.155018                0.039688          15.422325            1       True          3
3               XGBoost    0.932808   0.922980        0.434054       0.199338   33.613446                 0.434054                0.199338          33.613446            1       True         11
4            LightGBMXT    0.932338

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.946330,0.958141,2.095053,1.040706,367.465576,0.006284,0.002449,2.216262,2,True,14
1,NeuralNetTorch,0.945780,0.949738,0.043614,0.029637,21.725185,0.043614,0.029637,21.725185,1,True,12
2,NeuralNetFastAI,0.940793,0.952564,0.155018,0.039688,15.422325,0.155018,0.039688,15.422325,1,True,3
3,XGBoost,0.932808,0.922980,0.434054,0.199338,33.613446,0.434054,0.199338,33.613446,1,True,11
4,LightGBMXT,0.932338,0.925916,0.367487,0.118830,241.936384,0.367487,0.118830,241.936384,1,True,4
5,LightGBM,0.931289,0.914524,0.232640,0.103882,269.057187,0.232640,0.103882,269.057187,1,True,5
6,CatBoost,0.926701,0.908939,0.055557,0.040767,25.778524,0.055557,0.040767,25.778524,1,True,8
7,LightGBMLarge,0.904680,0.903231,0.267398,0.080914,621.820032,0.267398,0.080914,621.820032,1,True,13
8,RandomForestGini,0.889364,0.874945,0.177728,0.165718,54.403005,0.177728,0.165718,54.403005,1,True,6
9,ExtraTreesEntr,0.885373,0.869236,0.227469,0.163710,8.590983,0.227469,0.163710,8.590983,1,True,10


In [ ]:
%cd /content
predictor.save('clip_embed_predictor/predictor.pkl')

/content


In [ ]:
from autogluon.tabular.models import AbstractModel

In [ ]:
model = TabularPredictor.load('/content/AutogluonModels/ag-20230519_200219')

In [ ]:
model.leaderboard(test_data)

In [ ]:
import shutil
shutil.make_archive('clip_autogluon', 'zip', '/content/AutogluonModels/ag-20230519_200219')

'/content/clip_autogluon.zip'

In [7]:
timm_df = pd.read_json('/content/drive/MyDrive/convnext_xxlarge.clip_laion2b_embed.json') 

In [8]:
timm_df2 = pd.read_json('/content/drive/MyDrive/eva02_large_patch14_clip_336.merged2b_embed.json') 

In [114]:
timm_df_c  = (
    pd.concat([timm_df,timm_df2])
    .reset_index()
    .groupby("index")
    .mean()
)

<ipython-input-114-d9c39542d5f6>:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .mean()


In [ ]:
timm_df_c

In [113]:
timm_df_c.shape

(8964, 1001)

In [163]:
super_embed = pd.concat([df,
                         timm_df.drop(['indexes','cls'],axis=1),
                         timm_df2.drop(['indexes','cls'],axis=1),
                         #s_df.drop(['cls'],axis=1)
                         ],axis=1,ignore_index=True)

In [161]:
df.isnull().sum()

cls          0
indexes      0
embed_0      0
embed_1      0
embed_2      0
            ..
embed_763    0
embed_764    0
embed_765    0
embed_766    0
embed_767    0
Length: 770, dtype: int64

In [164]:
!rm -r /content/catboost_info
from catboost import CatBoostClassifier,Pool,cv
params = {'iterations':400,
         'loss_function':'MultiClass',
         'random_seed':42,
         'eval_metric':'TotalF1:average=Weighted',
         'task_type':'GPU'}
train_pool = Pool(data = super_embed.drop([0,1],axis=1),
                 label = super_embed[0])
cv_data = cv(train_pool,
            params = params,
            fold_count = 5,
            shuffle = True,
            stratified =True,
            verbose = False,
            seed = 42)

Training on fold [0/5]
bestTest = 0.9665719611
bestIteration = 394
Training on fold [1/5]
bestTest = 0.9693017235
bestIteration = 399
Training on fold [2/5]
bestTest = 0.9687154503
bestIteration = 356
Training on fold [3/5]
bestTest = 0.9598273178
bestIteration = 378
Training on fold [4/5]
bestTest = 0.9732000403
bestIteration = 392


In [ ]:
cv_data

,iterations,test-TotalF1:average=Weighted-mean,test-TotalF1:average=Weighted-std,train-TotalF1:average=Weighted-mean,train-TotalF1:average=Weighted-std,test-MultiClass-mean,test-MultiClass-std,train-MultiClass-mean,train-MultiClass-std
0,0,0.783331,0.014191,0.798884,0.005927,1.067108,0.000688,1.066097,0.000465
1,1,0.811545,0.015209,0.828390,0.006741,1.037377,0.000862,1.035471,0.000570
2,2,0.827067,0.009863,0.841591,0.010630,1.009053,0.000725,1.006508,0.000515
3,3,0.828403,0.013024,0.844849,0.008346,0.982991,0.001158,0.979435,0.000560
4,4,0.834939,0.010559,0.850846,0.010050,0.958027,0.000952,0.953895,0.000755
...,...,...,...,...,...,...,...,...,...
395,395,0.947858,0.006441,0.984376,0.000743,0.156364,0.006850,0.099819,0.001804
396,396,0.947747,0.006594,0.984376,0.000875,0.156250,0.006828,0.099655,0.001830
397,397,0.947630,0.006692,0.984403,0.000771,0.156154,0.006838,0.099488,0.001842
398,398,0.947740,0.006603,0.984459,0.000700,0.156035,0.006859,0.099296,0.001799


In [ ]:
cv_data['test-TotalF1:average=Weighted-mean'].max()

0.9481939404440538

# Autogluon

In [165]:
train_data,test_data = train_test_split(super_embed.drop([1],axis=1),train_size=0.8)

In [ ]:
predictor = TabularPredictor(label=0,eval_metric='f1_weighted').fit(train_data,num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels/ag-20230520_170832/"
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230520_170832/"
AutoGluon Version:  0.7.0
Python Version:     3.10.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Apr 29 09:15:28 UTC 2023
Train Data Rows:    7171
Train Data Columns: 2536
Label Column: 0
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	3 unique label values:  [0, 2, 1]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass',

In [ ]:
y_p = predictor.predict(test_data.drop([0],axis=1))

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_p,test_data[0],average='weighted')

0.9699087673688592

In [ ]:
predictor.leaderboard(test_data)

                  model  score_test  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        NeuralNetTorch    0.969857   0.980519        0.208574       0.161214    20.439817                 0.208574                0.161214          20.439817            1       True         12
1   WeightedEnsemble_L2    0.969857   0.980519        0.214383       0.162908    23.082009                 0.005809                0.001694           2.642191            2       True         14
2       NeuralNetFastAI    0.967627   0.980500        0.212771       0.044941    16.829405                 0.212771                0.044941          16.829405            1       True          3
3              CatBoost    0.962610   0.974930        0.313770       0.166630   118.232983                 0.313770                0.166630         118.232983            1       True          8
4            LightGBMXT    0.9

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetTorch,0.969857,0.980519,0.208574,0.161214,20.439817,0.208574,0.161214,20.439817,1,True,12
1,WeightedEnsemble_L2,0.969857,0.980519,0.214383,0.162908,23.082009,0.005809,0.001694,2.642191,2,True,14
2,NeuralNetFastAI,0.967627,0.980500,0.212771,0.044941,16.829405,0.212771,0.044941,16.829405,1,True,3
3,CatBoost,0.962610,0.974930,0.313770,0.166630,118.232983,0.313770,0.166630,118.232983,1,True,8
4,LightGBMXT,0.960382,0.966610,0.430904,0.108035,478.274311,0.430904,0.108035,478.274311,1,True,4
5,XGBoost,0.958679,0.966610,0.421402,0.212296,46.999780,0.421402,0.212296,46.999780,1,True,11
6,LightGBM,0.955332,0.966610,0.257324,0.092023,546.901360,0.257324,0.092023,546.901360,1,True,5
7,LightGBMLarge,0.947514,0.956767,0.862735,0.329193,1467.383089,0.862735,0.329193,1467.383089,1,True,13
8,RandomForestEntr,0.927408,0.928835,0.271753,0.113894,86.416256,0.271753,0.113894,86.416256,1,True,7
9,RandomForestGini,0.924032,0.924513,0.306012,0.152889,70.534253,0.306012,0.152889,70.534253,1,True,6


In [ ]:
train_data

,0,2,3,4,5,6,7,8,9,10,...,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769
8112,0,-0.413942,0.435494,-0.793888,-0.248041,-0.032162,-0.034451,-0.299811,-0.320214,0.085166,...,0.123449,0.770338,-0.102249,-0.015947,0.469086,-0.026474,0.923428,-0.583675,-0.359599,-0.165648
5369,2,-0.434096,0.905136,-0.638856,-0.030171,-0.305138,-0.094247,0.516428,-0.212246,0.536884,...,0.251008,0.129199,-0.007439,-0.141761,-0.186016,0.745377,0.195405,0.287446,0.080846,-0.290372
3768,2,-0.390327,0.350470,-0.588610,-0.284213,-0.000514,0.063387,0.320943,-0.243364,0.496425,...,-0.160748,0.180542,-0.092217,0.135577,-0.029392,0.351637,0.350605,-0.347354,-0.126755,0.019912
2076,1,-0.446708,0.349882,-0.312660,-0.357468,-0.260071,0.054856,0.487992,-0.257816,0.339567,...,0.143766,0.085777,-0.108464,0.219048,-0.056512,0.361471,0.129816,-0.610100,0.091250,0.290879
263,1,0.231999,0.427252,-0.158172,0.340994,-0.593554,-0.532577,0.566521,-0.463420,0.228979,...,0.165456,0.059883,0.016695,0.061188,0.075182,-0.091003,0.601884,0.240771,0.200379,-0.288484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,2,-0.447922,0.095834,-0.638063,-0.051293,-0.133408,0.082636,0.034153,-0.308446,0.223029,...,0.284310,0.284183,-0.130040,0.302741,-0.216126,1.131080,0.094611,-0.307942,0.295619,0.181117
7891,0,-0.366675,0.427856,-0.799431,0.047305,-0.058605,0.175410,-0.201314,-0.599897,-0.154555,...,0.316730,0.063796,0.203215,0.379898,0.475194,-0.315095,0.532730,-0.093904,0.143042,-0.080472
4859,2,-0.096339,0.826361,-0.227779,0.210653,-0.102717,-0.241900,0.251668,-0.224507,-0.079094,...,0.414994,-0.111904,-0.182740,-0.041773,-0.112598,0.382476,0.349130,-0.112907,0.084377,0.386288
3264,2,-0.069547,0.182496,-0.763187,0.410950,-0.668061,-0.134768,-0.171737,-0.089199,0.724893,...,0.120616,0.507855,-0.206666,0.352048,0.327630,0.898148,0.497709,-0.435758,0.265870,0.002601


In [130]:
import pickle
with open('/content/head_embeddings.pkl','rb') as f:
  head_embed_df = pd.read_pickle(f)

In [131]:
head_df = pd.DataFrame(head_embed_df)

ValueError: ignored

In [132]:
def pickle_to_df(vc):
  cls,embeds = [],[]
  for key in vc.keys():
    for emb in vc[key]:
      cls += [emb[0]]
      embeds += [emb[1]]
  df = pd.DataFrame()
  df['cls'] = cls
  df['embeds'] = embeds
  return df

In [86]:
def create_df(cl,embeds):
  df['cls'] = cl
  for i in range(embeds.shape[-1]):
    df[f'embed_{i}'] = embeds.T[i]
  return df

In [75]:
len(head_embed_df['klikun'])

2906

In [133]:
head_df = pickle_to_df(head_embed_df)

In [134]:
head_df.shape

(8710, 2)

In [ ]:
s_df = pd.DataFrame()
s_df['cls'] = head_df.cls
for i in range(stack_embeds.shape[-1]):
    s_df[f'embed_{i}'] = stack_embeds.T[i]

In [96]:
s_df

,cls,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,...,embed_758,embed_759,embed_760,embed_761,embed_762,embed_763,embed_764,embed_765,embed_766,embed_767
0,klikun,-0.069702,0.776942,-0.101612,-0.424751,0.007395,-0.311236,0.602114,-0.368819,-0.130258,...,-0.218638,-0.225793,0.340627,0.274738,-0.029114,0.351888,-0.113853,-0.349127,0.618051,-0.288561
1,klikun,-0.541816,-0.020332,-0.603222,-0.173064,-0.333344,-0.598795,0.374772,-0.427876,0.137932,...,-0.081952,0.577587,0.275515,0.073290,0.451168,0.059451,-0.225888,-0.224256,-0.152044,-0.670717
2,klikun,-0.033765,0.060185,0.196455,-0.423416,-0.015470,-0.085928,0.728862,-0.046027,0.348350,...,-0.023520,0.394046,0.864891,0.399384,-0.441592,-0.345249,0.158345,-0.152035,-0.072493,-0.390161
3,klikun,-0.057050,0.044154,-0.322684,-0.291424,-0.020998,-0.764601,0.524460,-0.332739,0.148689,...,-0.248146,0.005675,0.536797,0.237141,-0.257332,-0.029150,0.105368,-0.069256,0.306674,-0.669565
4,klikun,0.089616,0.064362,-0.005563,-0.927337,0.100691,-0.757027,0.673235,0.131409,0.078491,...,-0.421408,-0.243275,0.346853,0.039711,0.287310,0.295684,-0.099019,-0.106965,0.066363,-0.332298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8705,shipun,-0.191604,0.261954,-0.138735,-0.562072,0.061885,-0.181527,0.225430,-0.578328,0.102558,...,-0.039259,-0.310157,0.089441,-0.202056,-0.074921,0.156202,0.039738,-0.635091,-0.139590,0.028512
8706,shipun,0.099653,0.042099,-0.330253,-0.681073,0.027975,-0.761146,0.043960,-0.004363,-0.002586,...,-0.444129,-0.120973,0.214482,-0.139461,0.181656,-0.241008,-0.534225,-0.039703,0.146626,-0.518906
8707,shipun,-0.178926,0.427841,-0.378508,-0.071840,-0.061684,-0.337898,-0.072350,-0.277608,-0.171458,...,-0.028558,-0.265726,-0.307000,-0.111074,0.259078,0.602458,-0.072956,-0.248418,-0.345344,0.178724
8708,shipun,-0.555007,0.292875,-0.449270,-0.113568,-0.219130,-0.120549,0.216834,-0.819800,0.180421,...,-0.064392,-0.042015,-0.058439,0.005486,0.462947,0.349128,-0.019281,-0.255853,0.079908,-0.011276


In [92]:
s_df

,cls
0,klikun
1,klikun
2,klikun
3,klikun
4,klikun
...,...
8705,shipun
8706,shipun
8707,shipun
8708,shipun


In [87]:
create_df(head_df.cls,stack_embeds)

ValueError: ignored

In [78]:
stack_embeds = np.stack(head_df.embeds.tolist())

In [85]:
head_df = create_df(head_df.cls,stack_embeds)

ValueError: ignored

In [83]:
stack_embeds.T[0].shape

(8710,)

In [81]:
head_df.cls.shape

(8710,)